In [ ]:
import sys
import numpy as np
import math
from matplotlib import pyplot as plt
import matplotlib
import random
import os
from matplotlib import animation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy import spatial
from IPython.display import HTML
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel)

import importlib
import scripts.theodolite_function as tfu
import scripts.theodolite_utils as tfuu
tfu = importlib.reload(tfu)
from scipy.interpolate import splprep, splev
from scipy import interpolate
import seaborn as sns
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from scipy.spatial.transform import Rotation as R
import scripts.gp_prediction_utils as GPf
GPf = importlib.reload(GPf)
import GPy
from wbml.plot import tweak
from stheno import B, Measure, GP, EQ, Delta
import torch
from tqdm import tqdm

In [ ]:
def distance_time_trajectory(list_trajectories_split,trimble_1,time_trimble_1):
    distance = []
    time_travel = []

    for i in tqdm(list_trajectories_split):

        index_1 = np.array([i[0,0],i[1,0]])

        traj_1_x = np.atleast_2d(trimble_1[0, index_1[0]:index_1[1]+1]).T
        traj_1_y = np.atleast_2d(trimble_1[1, index_1[0]:index_1[1]+1]).T
        traj_1_z = np.atleast_2d(trimble_1[2, index_1[0]:index_1[1]+1]).T

        time_1 = time_trimble_1[index_1[0]:index_1[1]+1]

        dist = 0
        for j in range(0,len(traj_1_x)-1):
            point_before = np.array([traj_1_x[j],traj_1_y[j],traj_1_z[j]]) 
            point_after = np.array([traj_1_x[j+1],traj_1_y[j+1],traj_1_z[j+1]])  
            diff = point_before - point_after
            norm = np.linalg.norm(diff)
            dist = dist + norm

        distance.append(dist)
        time_travel.append(abs(time_1[-1]-time_1[0]))
    return sum(distance), sum(time_travel)

In [ ]:
%matplotlib notebook
tfuu = importlib.reload(tfuu)

list_dataset = [
                "./data/total_station/20220224/", 
                "./data/total_station/20220307/", 
                #"./data/total_station/20220312/",
                "./data/total_station/20220314/", 
                "./data/total_station/20220316/", 
                "./data/total_station/20220331-1/",
                "./data/total_station/20220331-2/", 
                #"./data/total_station/20220427/",
                "./data/total_station/20220505_cones/",
                "./data/total_station/20220505_empty/",
                "./data/total_station/20220513_evening_lidar/",
                "./data/total_station/20220513_evening_without_lidar_1/",
                "./data/total_station/20220513_evening_without_lidar_2/",
                "./data/total_station/20220513_morning_lidar/",
                "./data/total_station/20220513_morning_without_lidar_1/",
                "./data/total_station/20220513_morning_without_lidar_2/", 
                "./data/total_station/20220523_constrained/",
                "./data/total_station/20220523_cones/",
                "./data/total_station/20220523_empty/",
                "./data/total_station/20220523_other_tunnel/",
                "./data/total_station/20220525/01/",
                "./data/total_station/20220525/02/",
                "./data/total_station/20220622-1/",
                "./data/total_station/20220622-2/",
                "./data/total_station/20220630-1/",
                "./data/total_station/20220630-2/",
                "./data/total_station/20220711/01/",
                "./data/total_station/20220711/02/",
                "./data/total_station/20220715/01/",
                "./data/total_station/20220715/02/",
                "./data/total_station/20220715/03/",
                "./data/total_station/20220715/04/",
                #"./data/total_station/20220717/01/",
                "./data/total_station/20220717/02/",
                "./data/total_station/20220717/03/",
                "./data/total_station/20220717/04/",
                "./data/total_station/20220717/05/"

]

distance = []
time_experiment = []
for path in tqdm(list_dataset):

    time_trimble_1, trimble_1 = tfuu.read_point_data_csv_file(path+"trimble1.csv")
    time_trimble_2, trimble_2 = tfuu.read_point_data_csv_file(path+"trimble2.csv")
    time_trimble_3, trimble_3 = tfuu.read_point_data_csv_file(path+"trimble3.csv")

    tfu = importlib.reload(tfu)
    limit_time_interval = 4    #4 seconds max
    list_interval, list_time = tfu.split_time_interval_all_data(time_trimble_1, time_trimble_2, time_trimble_3, limit_time_interval)
    limit_search = 2
    list_trajectories_split = tfu.merge_interval(list_interval, list_time, time_trimble_1, time_trimble_2, time_trimble_3, limit_search)

    dist, time = distance_time_trajectory(list_trajectories_split,trimble_1,time_trimble_1)
    distance.append(dist)
    time_experiment.append(time)
print(sum(distance),sum(time_experiment))

In [ ]:
print(distance)
print(np.sum(distance))